# Readouts

All the models in this library are constructed around two main components:
- **core**: the core aims to (nonlinearly) extract features that are common between neurons. That is, we assume there exist a shared set of features that all neurons use but combine them in their own unique way.
- **readout**: once the core extracts the features, then we can predict neural activity by simply linearly combining those features into a single value.

By keeping the readout component of the network simple, during training we force most of the computations to be captured by the shared core.

While the core is shared among recording sessions, allowing to use more data for the shared representation learning, the  readout network (or networks), are different for each recording session, as each session will contain a different set of unique neurons we want to predict. 



In [1]:
import torch
from nnfabrik.builder import get_data
from nnfabrik.utility.nn_helpers import set_random_seed, get_dims_for_loader_dict
from neuralpredictors.utils import get_module_output

device = "cuda" if torch.cuda.is_available() else "cpu"
random_seed = 42

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
filenames = [
    '../../data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', 
    '../../data/static22846-10-16-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip'
    ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': True,
                 'batch_size': 32,
                 'scale':1,
                 'cuda': True if device == 'cuda' else False,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

In [3]:
model_config = {
    # core args
    'input_kern': 9,
    'hidden_kern': 7,
    'hidden_channels': 64,
    'layers': 4,
    'depth_separable': True,
    'stack': -1,
    'gamma_input': 6.3831,
}

Let us define a sample core network to use in the rest of the notebook

In [4]:
from neuralpredictors.layers.cores import Stacked2dCore

# We only need the train dataloaders to extract the session keys (could also use test or validation for this)
train_dataloaders = dataloaders["train"]

# Obtain the named tuple fields from the first entry of the first dataloader in the dictionary
example_batch = next(iter(list(train_dataloaders.values())[0]))
in_name, out_name = (
    list(example_batch.keys())[:2] if isinstance(example_batch, dict) else example_batch._fields[:2]
)

session_shape_dict = get_dims_for_loader_dict(train_dataloaders)
input_channels = [v[in_name][1] for v in session_shape_dict.values()]

core_input_channels = (
    list(input_channels.values())[0]
    if isinstance(input_channels, dict)
    else input_channels[0]
)

set_random_seed(random_seed)

core = Stacked2dCore(
    input_channels=core_input_channels,
    **model_config,
)


/usr/local/lib/python3.8/dist-packages/neuralpredictors/regularizers.py:153: UserWarning: LaplaceL2 Regularizer is deprecated. Use LaplaceL2norm instead.
  warnings.warn("LaplaceL2 Regularizer is deprecated. Use LaplaceL2norm instead.")
/usr/local/lib/python3.8/dist-packages/neuralpredictors/layers/cores/conv2d.py:127: UserWarning: The averaged value of regularizer will be used.
  warnings.warn("The averaged value of regularizer will be used.", UserWarning)


## Readout essentials

All readout networks in the package are initialised with two basic arguments, needed to properly initialised the readout learnable parameters:

- `in_shape`
- `outdims`
- `bias`

On top of this, if your dataset contains multiple sessions, you will need to use some instantiation of the `MultiReadoutBase` class, which creates multiple readouts for each session. Paralleling each individual readout instantiation, the multiple readouts takes as arguments:
- `in_shape_dict`, which will feed into each single readout `in_shape`
- `n_neurons_dict`, which will feed into each single readout `outdims`

The keys of both these dictionaries will be the session names, which are also passed in the forward method of this class to choose which individual readout will take care of the forward pass.

Let us create these two dictionaries before instantiating our readouts:


In [5]:
in_shapes_dict = {
    k: get_module_output(core, v[in_name])[1:]
    for k, v in session_shape_dict.items()
}
n_neurons_dict = {k: v[out_name][1] for k, v in session_shape_dict.items()}

## Factorised readout

https://papers.nips.cc/paper_files/paper/2017/hash/8c249675aea6c3cbd91661bbae767ff1-Abstract.html

In [6]:
from neuralpredictors.layers.readouts.factorized import FullFactorized2d
from neuralpredictors.layers.readouts.multi_readout import MultiReadoutBase

In [7]:
factorized_readout = MultiReadoutBase(
    in_shape_dict=in_shapes_dict,
    n_neurons_dict=n_neurons_dict,
    base_readout=FullFactorized2d,
    bias=True,
)

/usr/local/lib/python3.8/dist-packages/neuralpredictors/layers/readouts/base.py:88: UserWarning: Readout is NOT initialized with mean activity but with 0!
  warnings.warn("Readout is NOT initialized with mean activity but with 0!")


In [8]:
# The multi-readout layer is an instatiation of torch.ModuleDict, 
# so we can access the individual readouts by their session key

factorized_readout["21067-10-18"]

FullFactorized2d (64 x 144 x 256 -> 8372) with bias, normalized

In [9]:
factorized_readout["21067-10-18"]

FullFactorized2d (64 x 144 x 256 -> 8372) with bias, normalized

In [24]:
factorized_readout["21067-10-18"].weight

: 

In [10]:
factorized_readout 

MultiReadoutBase(
  (21067-10-18): FullFactorized2d (64 x 144 x 256 -> 8372) with bias, normalized
  (22846-10-16): FullFactorized2d (64 x 144 x 256 -> 7344) with bias, normalized
)

# Gaussian readout

https://www.biorxiv.org/content/10.1101/2020.10.05.326256v2

NameError: name 'MultipleFullGaussian2d' is not defined